In [3]:
from tensorflow.keras.models import load_model
from pydub import AudioSegment
import numpy as np
from pytube import YouTube 
from moviepy.video.io.VideoFileClip import VideoFileClip

In [4]:
model = load_model('../models/yt_cats_cnn_1')

In [9]:
def extract_audio_clip(video_path, file_name, start_time=None, end_time=None):
    ''' Take video path, start and end times as input and output .wav audio clip to the specified path'''
       
    # Load the video file
    video_clip = VideoFileClip(video_path)

    # Set the start and end times
    start_time = start_time or 0
    end_time = end_time or video_clip.duration

    # Get the audio from the video for the specified time interval
    audio_clip = video_clip.subclip(start_time, end_time).audio

    # Write the audio to a WAV file
    audio_clip.write_audiofile('../tmp_sliced_audio/'+ file_name)

    # Close the video and audio clips
    video_clip.close()
    audio_clip.close()

extract_audio_clip('../uploaded_video/court_tmo_interview.mp4', 'audio.wav', 20, 21)

MoviePy - Writing audio in ../tmp_sliced_audio/audio.wav


MoviePy - Done.


In [ ]:
def classify_audio_clips(clips):
    X = []
    for clip in clips:
        # Convert audio to spectrogram
        spectrogram = # Your code to generate spectrogram
        
        # Preprocess spectrogram
        preprocessed_spectrogram = # Your code to preprocess spectrogram
        
        X.append(preprocessed_spectrogram)
    
    X = np.array(X)
    
    # Run predictions
    y_pred = model.predict(X)
    
    # Get predicted classes
    predicted_classes = np.argmax(y_pred, axis=1)
    
    return predicted_classes


In [ ]:
def find_most_likely_clips(clips, predicted_classes, video_path):
    clips_by_category = {}
    for i, clip in enumerate(clips):
        category = predicted_classes[i]
        if category not in clips_by_category:
            clips_by_category[category] = []
        clips_by_category[category].append((clip, i))
    
    video = VideoFileClip(video_path)
    
    clips_with_video_timing = {}
    
    for category, clips in clips_by_category.items():
        clips_with_video_timing[category] = []
        for clip, clip_idx in clips:
            start_time = clip_idx * (clip.duration - overlap) / 1000
            end_time = (clip_idx + 1) * (clip.duration - overlap) / 1000
            video_clip = video.subclip(start_time, end_time)
            clips_with_video_timing[category].append(video_clip)
    
    return clips_with_video_timing

In [ ]:
def classify_video(video_path, clip_length, overlap):
    audio = extract_audio(video_path, 0, None)
    clips = generate_audio_clips(audio, clip_length, overlap)
    predicted_classes = classify_audio_clips(clips)
    most_likely_clips = find_most_likely_clips(clips, predicted_classes)
    return most_likely_clips